<a href="https://colab.research.google.com/github/amythemirror/Springboard/blob/master/API%20Mini-Project/api_data_wrangling_mini_project_method_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [0]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '********************'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [0]:
# First, import the relevant modules
import requests
import json
import operator
import statistics

In [0]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X'
params = dict(start_date='2020-04-28', end_date='2020-04-28', api_key=API_KEY)
res = requests.get(url, params=params)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(res.json())

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-04-24) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-05-08T22:16:12.693Z', 'newest_available_date': '2020-05-08', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2020-04-28', 'end_date': '2020-04-28', 'data': [['2020-04-28', 92.0, 92.2, 89.15, 89.65, None, 163949.0, 14804941.85, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

In [5]:
#collect data for the whole year 2017
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X'
params = dict(start_date='2017-01-01', end_date='2017-12-31', api_key=API_KEY)
res = requests.get(url, params=params)
print(res.status_code)

200


#### 2. Convert the returned JSON object into a Python dictionary.

In [6]:
#convert response into a json object
json_data = json.loads(res.text)

#create a nested dictionary
data = {}
for idx, daily_data in enumerate(json_data['dataset']['data']):
  data[idx] = {}
  for i in range(10):
    innerkeys = json_data['dataset']['column_names']
    data[idx][innerkeys[i]] = daily_data[i]

#check the first key-value pair
data[0]

{'Change': None,
 'Close': 51.76,
 'Daily Traded Units': None,
 'Date': '2017-12-29',
 'High': 51.94,
 'Last Price of the Day': None,
 'Low': 51.45,
 'Open': 51.76,
 'Traded Volume': 34640.0,
 'Turnover': 1792304.0}

#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [7]:
#extract opening prices into a list and skip None values
open_price = [v['Open'] for i, v in data.items() if v['Open'] != None]

#get max and min values of the list
max_open = max(open_price)
min_open = min(open_price)

print('The highest opening price is €%s' % max_open)
print('The lowest opening price is €%s' % min_open)

The highest opening price is €53.11
The lowest opening price is €34.0


#### 4. What was the largest change in any one day (based on High and Low price)?

In [0]:
#extract high and low prices into lists
high_price = [v['High'] for i, v in data.items()]
low_price = [v['Low'] for i, v in data.items()]

In [9]:
#calculate daily change for 2017 and get the max
change = list(map(operator.sub, high_price, low_price))
max_change = round(max(change), 2)
print('The largest change in any one day is €%s' % max_change)

The largest change in any one day is €2.81


#### 5. What was the largest change between any two days (based on Closing Price)?

In [0]:
#extract closing prices into lists
close_price = [v['Close'] for i, v in data.items()]

In [11]:
#calculate the largest change between any two days, and get the max
change = [round(abs(close_price[n] - close_price[n-1]), 2) for n in range(1, len(close_price))]

print('The largest change between any two days is €%s' % max(change))

The largest change between any two days is €2.56


#### 6. What was the average daily trading volume during this year?

In [0]:
#extract trading volumes into lists
trading_volume = [v['Traded Volume'] for i, v in data.items()]

In [13]:
#caculate the average daily trading volume
average = statistics.mean(trading_volume)

print('The average daily trading volume is %s' % average)

The average daily trading volume is 89124.33725490196


#### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [14]:
#method 1: define a function to find median
def median(numbers):
    if len(numbers) % 2 == 0:
        n = len(numbers)/2
        return (sorted(numbers)[n-1] + sorted(numbers)[n])/2
    else:
        return sorted(numbers)[len(numbers) // 2]

print('The median trading volume is %s' % median(trading_volume))

The median trading volume is 76286.0


In [15]:
#method 2: using statistics module
median = statistics.median(trading_volume)

print('The median trading volume is %s' % median)

The median trading volume is 76286.0
